In [2]:
from anthropic import Anthropic
from dotenv import load_dotenv


load_dotenv()

client = Anthropic()

In [4]:
# A relatively simple math problem
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"Multiply 1984135 by 9343116. Only respond with the result"}],
    max_tokens=400
)
print(response.content[0].text)

18,592,693,051,060


In [5]:
1984135 * 9343116

18538003464660

In [6]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    elif operation == "remainder":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero")
        return operand1 % operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

In [9]:
calculator_schema = {
    "name": "calculator",
    "description": "A basic calculator tool",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The mathematical operation that the calculator should execute",
                "enum": ["add", "subtract", "multiply", "divide", "remainder"],
            },
            "operand1": {
                "type": "number",
                "description": "The first number inputted in the calculator",
            },
            "operand2": {
                "type": "number",
                "description": "The second number inputted in the calculator",
            },
        },
        "required": ["operation", "operand1", "operand2"],
    }
}

In [10]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content": "Multiply 1984135 by 9343116. Only respond with the result"}],
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_schema]
)

In [11]:
response

Message(id='msg_01Arooz6PmeX1KYwmfZqUHp9', content=[ToolUseBlock(id='toolu_01PAF43HhXCrT45E8jcN45AM', input={'operand1': 1984135, 'operand2': 9343116, 'operation': 'multiply'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=430, output_tokens=93))

In [14]:
type(response.content[0])

anthropic.types.tool_use_block.ToolUseBlock

In [15]:
import anthropic

In [43]:
def execute_tool(tool_use_block):
    available_tools = {"calculator": calculator, "generate_wikipedia_reading_list": generate_wikipedia_reading_list}
    fn = available_tools[tool_use_block.name]
    return {
        "tool": tool_use_block.name,
        "input": tool_use_block.input,
        "result": fn(**tool_use_block.input)
    }

def query_with_tool_execution(**claude_kwargs):
    claude_response = client.messages.create(
        **claude_kwargs
    )
    formatted_response = []
    for content in claude_response.content:
        if isinstance(content, anthropic.types.tool_use_block.ToolUseBlock):
            formatted_response.append(execute_tool(content))
        else:
            formatted_response.append(content.to_dict())
    return formatted_response

In [35]:
query_with_tool_execution(
    model="claude-3-haiku-20240307",
    system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
    messages=[{"role": "user", "content": "I had 23 chickens but 2 flew away. How many are left?"}],
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_schema]
)

[{'text': "Okay, let's calculate this step-by-step:", 'type': 'text'},
 {'tool': 'calculator', 'result': 21}]

In [34]:
query_with_tool_execution(
    model="claude-3-haiku-20240307",
    system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
    messages=[{"role": "user", "content": "What is 201 times 2"}],
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_schema]
)

[{'text': "Okay, let's use the calculator tool to solve this:",
  'type': 'text'},
 {'tool': 'calculator', 'result': 402}]

In [33]:
query_with_tool_execution(
    model="claude-3-haiku-20240307",
    system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
    messages=[{"role": "user", "content": "Write me a haiku about bulgaria"}],
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_schema]
)

[{'text': "Here is a haiku about Bulgaria:\n\nRugged mountains rise,\nRhodope hills softly sway,\nBulgaria's charm.",
  'type': 'text'}]

In [24]:
tool_res = execute_tool_if_tool_request_made(response)

In [25]:
tool_res

18538003464660

In [46]:
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)
    return wikipedia_articles

def add_to_research_reading_file(articles, topic):
    with open("claude_tool_use_research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

In [38]:
generate_wikipedia_reading_list_schema = {
    "name": "generate_wikipedia_reading_list",
    "description": "This function expects to be passed a research topic like \"The history of Bulgaria\" or \"Climate in the 12th century\" and a list of potential Wikipedia article names. The function uses the wikipedia package to search for corresponding REAL wikipedia pages and builds a list of dictionaries that contain an article's title and URL.",
    "input_schema": {
        "type": "object",
        "properties": {
            "research_topic": {
                "type": "string",
                "description": "The broad research topic of the articles",
            },
            "article_titles": {
                "type": "array",
                "description": "List of potential wikipedia page names",
                "items": {
                    "type": "string",
                }
            },
        },
        "required": ["research_topic", "article_titles"],
    },
}

In [39]:
def get_research_help(topic: str, num_articles: int):
    res = query_with_tool_execution(
        model="claude-3-haiku-20240307",
        system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
        messages=[{"role": "user", "content": f"Come up with {num_articles} titles of potential Wikipedia articles related to the topic of {topic}. Then use the 'get_wikipedia_reading_list' function to populate the article URLs."}],
        max_tokens=300,
        # Tell Claude about our tool
        tools=[generate_wikipedia_reading_list_schema]
    )
    return res

In [48]:
get_research_help("Pirates Across The World", 7)

[{'text': 'Okay, let\'s generate a list of potential Wikipedia article titles related to the topic of "Pirates Across The World", and then use the provided tool to populate the article URLs:',
  'type': 'text'},
 {'tool': 'generate_wikipedia_reading_list',
  'input': {'research_topic': 'Pirates Across The World',
   'article_titles': ['History of Piracy',
    'Famous Pirates Throughout History',
    'Pirate Ships and Technology',
    'Pirate Treasure and Plunder',
    'Pirate Strongholds Around the Globe',
    'The Golden Age of Piracy',
    'Women Pirates in History']},
  'result': [{'title': 'Piracy',
    'url': 'https://en.wikipedia.org/wiki/Piracy'},
   {'title': 'Pittsburgh Pirates',
    'url': 'https://en.wikipedia.org/wiki/Pittsburgh_Pirates'},
   {'title': 'Piracy', 'url': 'https://en.wikipedia.org/wiki/Piracy'},
   {'title': 'Buried treasure',
    'url': 'https://en.wikipedia.org/wiki/Buried_treasure'},
   {'title': 'Piracy', 'url': 'https://en.wikipedia.org/wiki/Piracy'},
   

In [49]:
get_research_help("History of Hawaii", 3)

[{'text': 'Okay, here is a list of 3 potential Wikipedia article titles related to the history of Hawaii, along with their corresponding URLs:',
  'type': 'text'},
 {'tool': 'generate_wikipedia_reading_list',
  'input': {'research_topic': 'History of Hawaii',
   'article_titles': ['The Overthrow of the Hawaiian Kingdom',
    'History of Native Hawaiian Culture',
    'Colonization and Annexation of the Hawaiian Islands']},
  'result': [{'title': 'Overthrow of the Hawaiian Kingdom',
    'url': 'https://en.wikipedia.org/wiki/Overthrow_of_the_Hawaiian_Kingdom'},
   {'title': 'Culture of the Native Hawaiians',
    'url': 'https://en.wikipedia.org/wiki/Culture_of_the_Native_Hawaiians'},
   {'title': 'History of Hawaii',
    'url': 'https://en.wikipedia.org/wiki/History_of_Hawaii'}]}]

In [50]:
get_research_help("are animals conscious?", 3)

[{'text': "Okay, let's try to generate a Wikipedia reading list on the topic of whether animals are conscious:",
  'type': 'text'},
 {'tool': 'generate_wikipedia_reading_list',
  'input': {'research_topic': 'Are animals conscious?',
   'article_titles': ['Animal consciousness',
    'Theories of animal consciousness',
    'Evolutionary origins of animal consciousness']},
  'result': [{'title': 'Animal consciousness',
    'url': 'https://en.wikipedia.org/wiki/Animal_consciousness'},
   {'title': 'Animal consciousness',
    'url': 'https://en.wikipedia.org/wiki/Animal_consciousness'},
   {'title': 'Animal consciousness',
    'url': 'https://en.wikipedia.org/wiki/Animal_consciousness'}]}]